In [59]:
import json
from groq import Groq


In [ ]:
# validate json
def validate_json(raw_json):
    try:
        parsed = json.loads(raw_json)
        print("Valid JSON")
        print(parsed)
    except json.JSONDecodeError as e:
        print("Invalid JSON")
        print(e)


In [31]:
model_name = "llama-3.3-70b-versatile"

In [29]:
client = Groq() # GROQ_API_KEY

def call_groq(model_name, user_message):  
  completion = client.chat.completions.create(
    model=model_name,
    messages=[
      {
        "role": "user",
        "content": f"{user_message}"
      }
    ],
    temperature=0,
    max_completion_tokens=8192,
    top_p=1,
    # reasoning_effort="medium", # not supported for llama 3.3 70B
    stream=False,
    stop=None
  )

  return completion.choices[0].message

In [34]:
# Function testing

user_message = "what is BPE Algorithm? Explain under 50 words"
message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  BPE (Byte Pair Encoding) is a lossless text encoding algorithm that replaces frequent character pairs with a single token, reducing vocabulary size and improving language model efficiency.


Zer Shot Prompting

In [ ]:
user_message = """
Classify the text into neutral, negative or positive. 

Text: I thought the movie trailer would have more details.
Sentiment:
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
print("Reasoning: ", message.reasoning)

Reply:  Sentiment: Negative

The text expresses a sense of disappointment or unfulfilled expectation, which indicates a negative sentiment. The word "thought" implies that the speaker had certain expectations that were not met, and the phrase "would have more details" suggests that the trailer did not provide enough information, leading to a negative opinion.


In [35]:
user_message = "What is cricket?"

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  Cricket is a popular team sport played with a bat and ball on a rectangular field with two sets of three stumps (called wickets) at each end. The game is widely played in many countries, particularly in the Commonwealth nations, and is often referred to as the "gentleman's game."

Here's a brief overview of the game:

**Objective:**
The objective of cricket is to score runs by hitting the ball with a bat, while the opposing team tries to stop them by getting the batsmen out.

**The Game:**
A cricket match is typically played between two teams, each with 11 players. The game is divided into innings, with each team taking turns to bat and bowl.

1. **Batting:** The batting team sends two batsmen onto the field, who take turns to hit the ball bowled by the opposing team's bowler. The batsmen can score runs by running between the wickets or by hitting boundaries (four runs) or sixes (six runs).
2. **Bowling:** The bowling team has a bowler who delivers the ball towards the batsman.

Few Shot Prompting

In [36]:
user_message = """
iron -> metal
man -> human
iron man?
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  A clever play on words. 

If we follow the pattern:
iron -> metal
man -> human
Then, iron man would be... metal human.

Of course, this is a humorous interpretation, and in popular culture, Iron Man is a well-known superhero from the Marvel universe, played by Robert Downey Jr. But I see what you did there with the wordplay!


In [37]:
# trying with a different model

user_message = """
iron -> metal
man -> human
iron man?
"""

message = call_groq(model_name="qwen/qwen3-32b", user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  <think>
Okay, so the user is asking about "iron man" after giving the examples of "iron -> metal" and "man -> human". Let me break this down.

First, the examples are about categorization. Iron is a type of metal, and man is a type of human. So they're showing a hierarchical relationship where the first term is a specific instance of the second term. Now, applying that to "iron man", the user is probably expecting a similar categorization. But "Iron Man" is a known character, so maybe they want to know what category he fits into.

Wait, but maybe they're looking for a play on words. Like, if "iron" is a metal and "man" is a human, then "Iron Man" could be a human made of metal, which is exactly what the character is. But maybe they want a more abstract answer. Let me check if there's a standard answer for this kind of riddle.

Alternatively, could it be a trick question? Like, "Iron Man" is a specific person (Tony Stark), so maybe the answer is "human" but with a twist. But the

Chain-of-Thought (intermediate reasoning steps)

In [38]:
user_message = """
Find the x
1, 1, 2, 3, 4, 6, 9, 13, 18, x

with step by step reasoning
"""

message = call_groq(model_name="openai/gpt-oss-120b", user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  **Step 1 – Write the terms and the successive differences**

\[
\begin{array}{c|c|c}
n & a_n & \text{difference }d_n=a_n-a_{n-1}\\ \hline
1 & 1 & -\\
2 & 1 & d_2 = 1-1 = 0\\
3 & 2 & d_3 = 2-1 = 1\\
4 & 3 & d_4 = 3-2 = 1\\
5 & 4 & d_5 = 4-3 = 1\\
6 & 6 & d_6 = 6-4 = 2\\
7 & 9 & d_7 = 9-6 = 3\\
8 &13 & d_8 =13-9 = 4\\
9 &18 & d_9 =18-13 = 5\\
10& x & d_{10}=?
\end{array}
\]

So the list of differences is  

\[
0,\;1,\;1,\;1,\;2,\;3,\;4,\;5,\;\dots
\]

**Step 2 – Look for a pattern in the differences**

After the three consecutive 1’s the differences increase by 1 each step:

\[
\begin{aligned}
d_6 &= 2\\
d_7 &= 3\\
d_8 &= 4\\
d_9 &= 5\\
\text{next should be } d_{10} &= 6
\end{aligned}
\]

(The initial “0, 1, 1, 1” can be regarded as the start‑up of the same rule; from the 6th term onward the rule is simply “add 1 to the previous difference”.)

**Step 3 – Compute the next term**

\[
x = a_9 + d_{10}= 18 + 6 = 24.
\]

---

\[
\boxed{24}
\]


### Above reply formatted using markdown

Reply:  **Step 1 – Write the terms and the successive differences**

\[
\begin{array}{c|c|c}
n & a_n & \text{difference }d_n=a_n-a_{n-1}\\ \hline
1 & 1 & -\\
2 & 1 & d_2 = 1-1 = 0\\
3 & 2 & d_3 = 2-1 = 1\\
4 & 3 & d_4 = 3-2 = 1\\
5 & 4 & d_5 = 4-3 = 1\\
6 & 6 & d_6 = 6-4 = 2\\
7 & 9 & d_7 = 9-6 = 3\\
8 &13 & d_8 =13-9 = 4\\
9 &18 & d_9 =18-13 = 5\\
10& x & d_{10}=?
\end{array}
\]

So the list of differences is  

\[
0,\;1,\;1,\;1,\;2,\;3,\;4,\;5,\;\dots
\]

**Step 2 – Look for a pattern in the differences**

After the three consecutive 1’s the differences increase by 1 each step:

\[
\begin{aligned}
d_6 &= 2\\
d_7 &= 3\\
d_8 &= 4\\
d_9 &= 5\\
\text{next should be } d_{10} &= 6
\end{aligned}
\]

(The initial “0, 1, 1, 1” can be regarded as the start‑up of the same rule; from the 6th term onward the rule is simply “add 1 to the previous difference”.)

**Step 3 – Compute the next term**

\[
x = a_9 + d_{10}= 18 + 6 = 24.
\]

---

\[
\boxed{24}
\]


In [ ]:
user_message = "What is cricket?"

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  Sentiment: Negative

The text expresses a sense of disappointment or unfulfilled expectation, which indicates a negative sentiment. The word "thought" implies that the speaker had certain expectations that were not met, and the phrase "would have more details" suggests that the trailer did not provide enough information, leading to a negative opinion.


prompting to llm to give json output fails 50% of time

In [ ]:
user_message ="""
Extract the entities from the text below.

Text:
"Ajay is traveling from Bangalore to Kerala on 10 Dec."

Respond ONLY in valid JSON with exactly these keys:
name, source, destination, date

"""


client = Groq() # GROQ_API_KEY

completion = client.chat.completions.create(
model=model_name,
messages=[
    {
    "role": "user",
    "content": f"{user_message}"
    }
],
temperature=0,
max_completion_tokens=8192,
top_p=1,
# reasoning_effort="medium", # not supported for llama 3.3 70B
stream=False,
stop=None
)

completion.choices[0].message.content

'```\n{\n  "name": "Ajay",\n  "source": "Bangalore",\n  "destination": "Kerala",\n  "date": "10 Dec"\n}\n```'

{\n  "name": "Ajay",\n  "source": "Bangalore",\n  "destination": "Kerala",\n  "date": "10 Dec"\n}

In [58]:
validate_json(completion.choices[0].message.content)

Invalid JSON
Expecting value: line 1 column 1 (char 0)


In [46]:
user_message ="""
Extract the entities from the text below.

Text:
"Ajay is traveling from Bangalore to Kerala on 10 Dec."

Respond ONLY in valid JSON with exactly these keys:
name, source, destination, date

"""


client = Groq() # GROQ_API_KEY

completion = client.chat.completions.create(
model=model_name,
messages=[
    {
    "role": "user",
    "content": f"{user_message}"
    }
],
temperature=0,
max_completion_tokens=8192,
top_p=1,
# reasoning_effort="medium", # not supported for llama 3.3 70B
stream=False,
stop=None,
response_format={"type": "json_object"}
)

completion.choices[0].message.content

'{\n  "name": "Ajay",\n   "source": "Bangalore",\n   "destination": "Kerala",\n   "date": "10 Dec"\n}'

{\n  "name": "Ajay",\n   "source": "Bangalore",\n   "destination": "Kerala",\n   "date": "10 Dec"\n}

In [56]:
validate_json(completion.choices[0].message.content)

Valid JSON
{'name': 'Ajay', 'source': 'Bangalore', 'destination': 'Kerala', 'date': '10 Dec'}


In [61]:
from pydantic import BaseModel

client = Groq()

class ValidationStatus(BaseModel):
    is_valid: bool
    syntax_errors: list[str]

class SQLQueryGeneration(BaseModel):
    query: str
    query_type: str
    tables_used: list[str]
    estimated_complexity: str
    execution_notes: list[str]
    validation_status: ValidationStatus

response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct-0905",
    messages=[
        {
            "role": "system",
            "content": "You are a SQL expert. Generate structured SQL queries from natural language descriptions with proper syntax validation and metadata. Don't include any escape sequences like \n, \t, and all",
        },
        {"role": "user", "content": "Find all customers who made orders over $500 in the last 30 days, show their name, email, and total order amount"},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "sql_query_generation",
            "schema": SQLQueryGeneration.model_json_schema()
        }
    }
)

sql_query_generation = SQLQueryGeneration.model_validate(json.loads(response.choices[0].message.content))
print(json.dumps(sql_query_generation.model_dump(), indent=2))

{
  "query": "SELECT c.name, c.email, SUM(o.amount) AS total_order_amount\nFROM customers c\nJOIN orders o ON c.customer_id = o.customer_id\nWHERE o.order_date >= CURRENT_DATE - INTERVAL '30 days'\nGROUP BY c.customer_id, c.name, c.email\nHAVING SUM(o.amount) > 500;",
  "query_type": "SELECT",
  "tables_used": [
    "customers",
    "orders"
  ],
  "estimated_complexity": "Medium",
  "execution_notes": [
    "Ensure order_date column is of DATE or TIMESTAMP type",
    "Adjust interval syntax if database is not PostgreSQL",
    "Verify amount column exists in orders table"
  ],
  "validation_status": {
    "is_valid": true,
    "syntax_errors": []
  }
}


In [63]:
sql_query_generation.query

"SELECT c.name, c.email, SUM(o.amount) AS total_order_amount\nFROM customers c\nJOIN orders o ON c.customer_id = o.customer_id\nWHERE o.order_date >= CURRENT_DATE - INTERVAL '30 days'\nGROUP BY c.customer_id, c.name, c.email\nHAVING SUM(o.amount) > 500;"

In [71]:
# trying with a different model

user_message = """
You're a wage-worker of 1900s. Explain the significance of the Industrial Revolution.
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  (sigh) Ah, 'ello there. I'm just a simple fella, workin' at the textile mill down the street. Been doin' it for years, since I was a young'un. You're askin' about this "Industrial Revolution" business, eh? Well, let me tell you, it's changed everything.

Used to be, we made things by hand, or with the help of simple tools. My dad was a blacksmith, and he'd spend hours hammerin' away at a piece of hot metal to make a single tool. But now, with all these new machines and factories, we can make things faster and cheaper than ever before. It's like magic, I tell ya!

The Industrial Revolution, it's brought us all sorts of new technologies: steam engines, power looms, and them newfangled machines that can do the work of a dozen men. It's made it possible for us to produce more goods than we ever thought possible. And that's created all sorts of new jobs, like mine at the mill.

But it's not all good, if you know what I mean. The work's hard, and the hours are long. I'm talkin' 12 ho

In [70]:
# trying with a different model

user_message = """
You are a historian. Explain the significance of the Industrial Revolution.
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)
# print("Reasoning: ", message.reasoning)

Reply:  The Industrial Revolution was a transformative period in human history, marking the shift from manual labor to machine-based manufacturing and having a profound impact on the social, economic, and cultural fabric of societies around the world. The significance of the Industrial Revolution can be understood from several perspectives:

1. **Economic Transformation**: The Industrial Revolution marked the transition from a primarily agrarian and handicraft-based economy to one dominated by industry and machine-based manufacturing. New technologies, such as the steam engine and power looms, enabled mass production, increased efficiency, and reduced costs, leading to unprecedented economic growth and the emergence of new industries.
2. **Urbanization and Demographic Changes**: As people moved from rural areas to cities in search of work in factories, urban populations grew rapidly, leading to the development of new social systems, including the growth of a industrial working class. T

In [ ]:
# easy to understand context and question

user_message = """
Answer the question using ONLY the provided context.

Context:
"Python was created by Guido van Rossum."

Question:
In which year was Python first released?

If not in context, respond exactly: UNKNOWN
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)

Reply:  UNKNOWN


In [ ]:
# ask a question with well known answer, then prompting fails. 

user_message = """
Answer the question using ONLY the provided context.

Context:
"A software without any failure point is a good software.
In python exception-handling strategies is the best way to make good softwares."

Question:
In Python typically with 'try' which keyword is to be used strictly? 

If not in context, respond exactly: UNKNOWN
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)

Reply:  Except


In [ ]:
# grounded extraction test (RAG-style)

user_message = """
Extract facts ONLY from the text.

Text:
"The Mars rover Perseverance landed in 2021."

Question:
Which rocket launched Perseverance?

If not stated, respond: NOT STATED
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)

Reply:  NOT STATED


In [ ]:
# Stress Test (Role + Safety)

user_message = """
You are a medical guideline summarization assistant.

Rules:
- Use ONLY the provided excerpt
- If information is missing, say "NOT IN GUIDELINES"
- Do not infer or generalize

Excerpt:
"Paracetamol is recommended for mild pain."

Question:
What dosage should be used in children?
"""

message = call_groq(model_name=model_name, user_message=user_message)

print("Reply: ", message.content)

Reply:  NOT IN GUIDELINES
